# **Burger's equation**:
# $ u_t + u u_{xx} = v u_{xx} \qquad x\in[-1,1], \quad t\in [0,1], \quad u ≡ u(x,t)$

## Boundary conditions (Dirichlet):
* $ u(-1, t) = 0, \qquad t\in [0,1]$
* $ u(1, t) = 0, \qquad t\in [0,1]$

## Initial condition:
* $ u(x, 0) = -\sin(\pi x), \qquad x\in[-1,1] $

<!-- ## *ANALYTICAL SOLUTION*
## $ u(x, y, t) = e^{-13\pi^2t}\sin(3\pi x)\sin(2\pi y)  $ -->




# Installations

In [1]:
!pip install -Uqq ipdb
!pip install torch==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incomp

# Imports

In [2]:
import math
import warnings
import torch
import ipdb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn

from tqdm.auto import tqdm
from google.colab import drive

warnings.filterwarnings("ignore")
drive.mount('/content/drive')

print(torch.__version__)

Mounted at /content/drive
1.13.1+cu117


# Helpers

## Functions

In [47]:
def generate_random_numbers(min_, max_, N, dtype):
    return min_ + (max_ - min_) * torch.rand(size=(N, 1), dtype=dtype)


def format_loss(loss):
    if loss == 0:
        return "0.0e+00"

    # Calculate the exponent part
    exponent = int(math.log10(abs(loss)))

    # Determine the format based on the value of the loss
    if abs(loss) < 1:
        formatted_loss = f"{loss:.2e}"
    else:
        # Adjust the sign of the formatted loss
        sign = "-" if loss < 0 else ""

        # Calculate the number of decimal places
        decimal_places = 2 - exponent

        # Ensure at least two decimal places
        decimal_places = max(decimal_places, 2)

        # Format the loss with the correct sign
        formatted_loss = f"{sign}{abs(loss):.{decimal_places}e}"

    return formatted_loss


def grad(output, input):
    return torch.autograd.grad(
                output, input,
                grad_outputs=torch.ones_like(output),
                retain_graph=True,
                create_graph=True
            )[0]


def calculate_pde(x, u, v):
    # Calculate 1st and 2nd derivatives
    du_dX = grad(u, x)
    du_dXX = grad(du_dX, x)

    # Retrieve the partial gradients
    du_dt = du_dX[:, 1].flatten()
    du_dx = du_dX[:, 0].flatten()
    du_dxx = du_dXX[:, 0].flatten()

    return du_dt + u.flatten() * du_dx - v * du_dxx


def checkpoint(net, checkpoint_path):
    torch.save(
        {"net": net.state_dict()},
        checkpoint_path
    )


def keep_checkpoints_and_print_losses(iter, loss, loss_test, vars_to_track, constant_vars):
    # ANSI escape sequence for bold text
    bold = "\033[1m"

    # Reset ANSI escape sequence
    reset = "\033[0m"

    loss_str = format_loss(loss)
    loss_test_str = format_loss(loss_test)

    if iter == 1:
        vars_to_track["flag"] = True
        vars_to_track["best_iter"] = -1
        vars_to_track["best_val_loss"] = loss_test
        checkpoint(vars_to_track["net"], constant_vars["checkpoint_path"])
        print(f"{bold}Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str} - *Checkpoint*{reset}")
    else:
        if loss_test < vars_to_track["best_val_loss"]:
            vars_to_track["flag"] = True
            vars_to_track["best_iter"] = iter
            vars_to_track["best_val_loss"] = loss_test
            checkpoint(vars_to_track["net"], constant_vars["checkpoint_path"])

            if iter % constant_vars["print_every"] == 0:
                print(f"{bold}Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str} - *Checkpoint*{reset}")

        elif iter - vars_to_track["best_iter"] > constant_vars["patience"]:
            if iter % constant_vars["print_every"] == 0:
                vars_to_track["early_stopping_applied"] = True
                print(f"{bold}Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str}{reset}")
            return vars_to_track

    if (vars_to_track["flag"] is False) and (iter % constant_vars["print_every"] == 0):
        print(f"{bold}Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str}{reset}")

    return vars_to_track


def print_training_info(iters, num_epochs_adam, num_epochs_lbfgs, resampling_per_iters):
    if resampling_per_iters == 0:
        print(f"PINN: {iters} iterations (NO resampling)")
    else:
        print(f"PINN: {iters} iterations (Resampling every {resampling_per_iters} iterations)")
    print(f"a. {num_epochs_adam} epochs --> Adam")
    print(f"b. {num_epochs_lbfgs} epochs --> L-BFGS")


def print_time_elapsed(start, end):
    dt = pd.Timedelta(end - start)
    seconds = dt.seconds
    minutes = seconds / 60.
    hours = seconds / 3600.
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))


def rar_sampling_strategy(pde_res, domain_data, num_max_errors=1):
    domain_points = domain_data.detach().cpu().numpy()
    abs_errors = torch.abs(pde_res).detach().cpu().numpy().flatten()
    df = pd.DataFrame(
        {
            "x": domain_points[:, 0].flatten(),
            "t": domain_points[:, 1].flatten(),
            "abs_errors": abs_errors,
        }
    )
    df.sort_values(by='abs_errors', ascending=False, inplace=True)
    return df.head(num_max_errors)[["x", "t"]].to_numpy()

## Classes

### Data

In [48]:
class Data():
    def __init__(self, dtype, device):
        self.dtype = dtype
        self.device = device

    # *** Create in-domain points ***
    def sample_domain(self, data_config):
        # Random Grid
        x_domain = generate_random_numbers(data_config["x_min"], data_config["x_max"], data_config["Nx_domain"], data_config["data_type"])
        t_domain = generate_random_numbers(data_config["t_min"], data_config["t_max"], data_config["Nt_domain"], data_config["data_type"])
        domain_data = torch.cat((x_domain, t_domain), dim=1)

        return domain_data.to(device=data_config["device"]).requires_grad_(True)

    # *** Boundary Conditions ***
    def sample_boundary(self, data_config):
        # Random boundary points
        t_bound = generate_random_numbers(data_config["t_min"], data_config["t_max"], data_config["Nt_bound"], data_config["data_type"])
        x_left = - torch.ones(len(t_bound), 1, dtype=data_config["data_type"])
        x_right = torch.ones(len(t_bound), 1, dtype=data_config["data_type"])

        bound_data_left = torch.cat((x_left, t_bound), dim=1)
        bound_data_right = torch.cat((x_right, t_bound), dim=1)
        bound_data = torch.cat((bound_data_left, bound_data_right), dim=0).to(data_config["device"])
        u_bound = torch.zeros(len(bound_data), 1, dtype=data_config["data_type"], device=data_config["device"])

        return bound_data, u_bound

    # *** Initial Condition ***
    def sample_initial(self, data_config):
        # Random initial points
        x_init = generate_random_numbers(data_config["x_min"], data_config["x_max"], data_config["Nx_init"], data_config["data_type"])
        t_init = torch.zeros(len(x_init), 1, dtype=data_config["data_type"])
        init_data = torch.cat((x_init, t_init), dim=1).to(data_config["device"])
        u_init = - torch.sin(math.pi * x_init).to(data_config["device"])

        return init_data, u_init

    # *** Train set ***
    def generate_train_data(self, data_config):
        train_data = {}
        train_data["domain_data"] = self.sample_domain(data_config)
        train_data["bound_data"], train_data["u_bound"] = self.sample_boundary(data_config)
        train_data["init_data"], train_data["u_init"] = self.sample_initial(data_config)

        return train_data

    # *** Test set ***
    def get_test_data(self, data_config):
        test_data = pd.read_csv(data_config["test_dir"]).dropna().to_numpy()
        return torch.tensor(test_data, dtype=data_config["data_type"], device=data_config["device"], requires_grad=True)

### Networks

In [49]:
class MLP(nn.Module):
    def __init__(self, mlp_config):
        super().__init__()

        self.layers = mlp_config["mlp_layers"]
        self.activation = mlp_config["mlp_hidden_activation"]
        self.weight_init = mlp_config["mlp_weight_init"]
        self.bias_init = mlp_config["mlp_bias_init"]
        self.device = mlp_config["mlp_device"]

        dense_layers = [
            self._dense_layer(in_features=self.layers[i], out_features=self.layers[i + 1])
            for i in range(len(self.layers) - 2)
        ]
        dense_layers.append(nn.Linear(in_features=self.layers[-2], out_features=self.layers[-1]))

        self.mlp = nn.Sequential(*dense_layers).to(self.device)

    def _dense_layer(self, in_features, out_features):
        dense_layer = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=out_features),
        )

        if self.weight_init is not None:
            self.weight_init(dense_layer[0].weight)

        if self.bias_init is not None:
            self.bias_init(dense_layer[0].bias)

        dense_layer.add_module("activation", self.activation)
        return dense_layer

    def forward(self, x):
        return self.mlp(x)

### PINN-Base

In [50]:
class PINNBase():
    def __init__(self, base_pinn_config):

        # Constants
        self.v = base_pinn_config["v"]

        # Define the model
        self.net = base_pinn_config["net"]

        # Set the optimizers
        self.optimizers = {
            "adam": base_pinn_config["adam_optimizer"],
            "lbfgs": base_pinn_config["lbfgs_optimizer"]
        }

        # Set the Loss function
        self.criterion = nn.MSELoss()

        # Set the MAE criterion (used for test data only)
        self.l1_loss = nn.L1Loss()

    def calculate_pde_loss(self, data):
        # Calculate the domain loss
        u_hat = self.net(data)
        self.pde_res = calculate_pde(data, u_hat, self.v)
        pde_target = torch.zeros_like(self.pde_res)
        return self.criterion(self.pde_res, pde_target)

    def calculate_total_loss(self, data):
        # Calculate boundary loss
        loss_b = self.criterion(
            self.net(data["bound_data"]).flatten(),
            data["u_bound"].flatten()
        )

        # Calculate initial loss
        loss_i = self.criterion(
            self.net(data["init_data"]).flatten(),
            data["u_init"].flatten()
        )

        # Calculate the domain loss
        loss_pde = self.calculate_pde_loss(data["domain_data"])

        # Calculate total discriminator loss
        return loss_b + loss_i + loss_pde

    def evaluate_pinn(self, test_data):
        u_hat = self.net(test_data)
        pde_res = calculate_pde(test_data, u_hat, self.v)
        pde_target = torch.zeros_like(pde_res)
        return self.l1_loss(pde_res, pde_target)

### PINN

In [51]:
class PINN():
    def __init__(self, data_init, pinn_config):
        # Data
        self.data_init = data_init

        # Base PINN
        self.base_pinn = pinn_config["base_pinn"]

        # Constants
        self.stars = 30 * "*"
        self.checkpoint_path = pinn_config["checkpoint_path"]
        self.dtype = self.data_init.dtype
        self.device = self.data_init.device


    def _train_step(self):
        loss = self.base_pinn.calculate_total_loss(self.train_data)
        if torch.isnan(loss).item():
            return loss
        loss.backward()
        return loss

    def _closure(self):
        self.optimizer.zero_grad()
        return self._train_step()

    def _train_with_adam(self, num_epochs_adam):
        self.optimizer = self.base_pinn.optimizers['adam']
        print("a. Training with Adam...")
        for epoch in tqdm(range(1, num_epochs_adam + 1)):
            self.optimizer.zero_grad()
            loss = self._train_step()
            self.optimizer.step()

    def _train_with_lbfgs(self, num_epochs_lbfgs):
        self.optimizer = self.base_pinn.optimizers["lbfgs"]
        print("\nb. Training with L-BFGS...\n")
        for epoch in range(1, num_epochs_lbfgs + 1):
            loss = self.optimizer.step(self._closure)
        return loss

    def train(self, data_config, train_config, use_resampling=False,
              resampling_per_iters=10, num_max_errors=None):

        # 'Zero' resampling_per_iters means no resampling
        if resampling_per_iters == 0:
            use_resampling = False

        # Get test set
        print("Getting test set...\n")
        test_data = self.data_init.get_test_data(data_config)

        # Generate train set before training starts
        # ONLY if no resampling is applied
        if use_resampling is False:
            print("Sampling training set...\n")
            self.train_data = self.data_init.generate_train_data(data_config)

        # Configure the necessary variables properly
        iters = train_config["iterations"]
        num_epochs_adam = train_config["num_epochs_adam"]
        num_epochs_lbfgs = train_config["num_epochs_lbfgs"]

        vars_to_track = {"net": self.base_pinn.net}

        constant_vars = {
            "patience": train_config['patience'],
            "print_every": train_config['print_every'],
            "checkpoint_path": self.checkpoint_path,
        }

        # Print number of iterations and number of epochs per optimizer used during training
        print_training_info(iters, num_epochs_adam, num_epochs_lbfgs, resampling_per_iters)

        # Start counting training time
        t0 = pd.Timestamp.now()

        for iter in tqdm(range(1, iters + 1)):
            print(f"\n{self.stars} Iteration {iter} {self.stars}")

            vars_to_track["early_stopping_applied"] = False
            vars_to_track["flag"] = False

            if use_resampling and ((iter-1) % resampling_per_iters == 0):
                if iter == 1:
                    print("Sampling training set...\n")
                else:
                    print("Re-sampling training set...\n")
                self.train_data = self.data_init.generate_train_data(data_config)

            # Add points for RAR and Multi-Sampling RAR
            if (num_max_errors is not None) and (iter > 1):
                self.train_data["domain_data"] = torch.cat((self.train_data["domain_data"], rar_points), dim=0)

            # Train with Adam
            _ = self._train_with_adam(num_epochs_adam)

            # Train with L-BFGS
            loss = self._train_with_lbfgs(num_epochs_lbfgs)

            # Evaluate PINN on test set
            loss_test = self.base_pinn.evaluate_pinn(test_data)

            # Check for NaN loss (due to L-BFGS) and skip rest of the loop in this case
            if torch.isnan(loss).item():
                print("\nDetected NaN training loss. Skipping to next iteration...")
                continue
            if torch.isnan(loss_test).item():
                print("\nDetected NaN test loss. Skipping to next iteration...")
                continue

            # Keep checkpoints, print losses and update necessary variables
            vars_to_track = keep_checkpoints_and_print_losses(iter, loss, loss_test, vars_to_track, constant_vars)

            if vars_to_track["early_stopping_applied"]:
                print(f"\nEarly stopping applied at epoch {iter}.")
                print(f"TOTAL TRAINING TIME: ")
                print_time_elapsed(t0, pd.Timestamp.now())
                break

            # Find points with the greatest errors
            if num_max_errors is not None:
                points_to_add = rar_sampling_strategy(pde_res=base_pinn.pde_res,
                                                      domain_data=self.train_data["domain_data"],
                                                      num_max_errors=num_max_errors)
                rar_points = torch.tensor(points_to_add, dtype=self.dtype, device=self.device, requires_grad=True)
                all_rar_points = rar_points if iter == 1 else torch.cat((all_rar_points, rar_points), dim=0)

# Initialization (**all experiments should have the same initialization!**)

## General Configuration (common to all experiments)

In [52]:
# Basic
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
dtype = torch.float32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Data
N_domain = 2_500
N_bound = 50
N_init = 100

data_config = {
    "x_min": -1.0,
    "x_max": 1.0,
    "t_min": 0.0,
    "t_max": 1.0,
    "Nx_domain": N_domain,
    "Nt_domain": N_domain,
    "Nx_init": N_init,
    "Nt_bound": N_bound,
    "test_dir": '/content/drive/MyDrive/test_data.csv',
    "device": device,
    "data_type": dtype,
}

# MLP
mlp_layers = 3
mlp_neurons_per_layer = 20
mlp_hidden_activation = nn.Tanh()
mlp_bias_init = lambda m: nn.init.zeros_(m.data)

if mlp_hidden_activation == nn.Tanh():
    mlp_weight_init = lambda m: nn.init.xavier_normal_(m.data, nn.init.calculate_gain('tanh'))
else:
    mlp_weight_init = lambda m: nn.init.xavier_normal_(m.data)

mlp_config = {
    "mlp_layers": [2] + mlp_layers * [mlp_neurons_per_layer] + [1],
    "mlp_hidden_activation": mlp_hidden_activation,
    "mlp_weight_init": mlp_weight_init,
    "mlp_bias_init": mlp_bias_init,
    "mlp_device": device,
}

# PINN-Base
adam_learning_rate = 1e-3
adam_weight_decay = 0
lbfgs_learning_rate = 1.0
lbfgs_max_iterations = 1_500
lbfgs_max_eval = 1_500 * 1.25

# Training
num_epochs_adam = 1_000
num_epochs_lbfgs = 1

## Make the appropriate initializations...

In [53]:
# Initialize data
data_init = Data(dtype, device)

# Initialize PINN's network
mlp = MLP(mlp_config)

# Set optimizers
adam_optim = torch.optim.Adam(mlp.parameters(), lr=adam_learning_rate, weight_decay=adam_weight_decay)

lbfgs_optim = torch.optim.LBFGS(mlp.parameters(),
                                lr=lbfgs_learning_rate,
                                max_iter=lbfgs_max_iterations,
                                max_eval=lbfgs_max_eval,
                                tolerance_grad=1e-07,
                                tolerance_change=1e-09,
                                history_size=100,
                                line_search_fn='strong_wolfe')

# Base PINN configuration
base_pinn_config = {
    "v": 0.01 / math.pi,
    "net": mlp,
    "adam_optimizer": adam_optim,
    "lbfgs_optimizer": lbfgs_optim,
}

# Create base PINN
base_pinn = PINNBase(base_pinn_config)

# PINN configuration
pinn_config = {
    "base_pinn": base_pinn,
    "checkpoint_path": "pinn.pth",
}

# Create PINN
pinn = PINN(data_init, pinn_config)

# **A) Reference models**

## 1. Baseline

In [54]:
# Training configuration
iters = 100
patience = iters
use_resampling = False
resampling_per_iters = 0
num_max_errors = None

train_config = {
    "iterations": iters,
    "patience": patience,
    "print_every": 1,
    "num_epochs_adam": num_epochs_adam,
    "num_epochs_lbfgs": num_epochs_lbfgs,
}

# Train baseline
pinn.train(data_config=data_config,
           train_config=train_config,
           use_resampling=use_resampling,
           resampling_per_iters=resampling_per_iters,
           num_max_errors=num_max_errors)

Getting test set...

Sampling training set...

PINN: 100 iterations (NO resampling)
a. 1000 epochs --> Adam
b. 1 epochs --> L-BFGS


  0%|          | 0/100 [00:00<?, ?it/s]


****************************** Iteration 1 ******************************
a. Training with Adam...


  0%|          | 0/1000 [00:00<?, ?it/s]


b. Training with L-BFGS...

Iteration: 1 | loss: 1.04e-01 | test_mae: 1.66e-02 - *Checkpoint*

****************************** Iteration 2 ******************************
a. Training with Adam...


...


b. Training with L-BFGS...


Detected NaN training loss. Skipping to next iteration...

****************************** Iteration 99 ******************************
a. Training with Adam...


  0%|          | 0/1000 [00:00<?, ?it/s]


b. Training with L-BFGS...


Detected NaN training loss. Skipping to next iteration...

****************************** Iteration 100 ******************************
a. Training with Adam...


  0%|          | 0/1000 [00:00<?, ?it/s]


b. Training with L-BFGS...


Detected NaN training loss. Skipping to next iteration...


## 2. Random Resampling

In [ ]:
# Training configuration
iters = 10
patience = 7    # iters
use_resampling = True
resampling_per_iters = 3

train_config = {
    "iterations": iters,
    "patience": patience,
    "print_every": 1,
    "num_epochs_adam": num_epochs_adam,
    "num_epochs_lbfgs": num_epochs_lbfgs,
}

# Train baseline
pinn.train(data_config=data_config,
           train_config=train_config,
           use_resampling=use_resampling,
           resampling_per_iters=resampling_per_iters)

In [ ]:
# ANSI escape sequence for bold text
bold = "\033[1m"

# Reset ANSI escape sequence
reset = "\033[0m"

# Sample variables
variable1 = "Hello"
variable2 = "World"

# Print statement with variables in bold
print(f"{bold}{variable1} {variable2}{reset}")


## 3. Residual Adaptive Resampling (RAR)

In [ ]:
%pdb on

In [ ]:
# Training configuration
iters = 100
patience = iters
use_resampling = False
resampling_per_iters = 0
num_max_errors = 1

train_config = {
    "iterations": iters,
    "patience": patience,
    "print_every": 1,
    "num_epochs_adam": num_epochs_adam,
    "num_epochs_lbfgs": num_epochs_lbfgs,
}

# Train baseline
pinn.train(data_config=data_config,
           train_config=train_config,
           use_resampling=use_resampling,
           resampling_per_iters=resampling_per_iters,
           num_max_errors=num_max_errors)

In [ ]:
# train_df = pd.DataFrame(pinn.train_data["domain_data"].detach().cpu().numpy())
# train_df.isna().sum()

In [ ]:
# params_list = []
# for _, param in pinn.base_pinn.net.named_parameters():
#     params_list.append(params_list)

# params_df = pd.DataFrame(np.array(params_list))
# params_df.isna().sum()

In [ ]:
# pinn.train(data_config=data_config,
#            train_config=train_config,
#            use_resampling=use_resampling,
#            resampling_per_iters=resampling_per_iters,
#            num_max_errors=num_max_errors)

## 4. Multi-Sampling RAR

In [ ]:
# Training configuration
iters = 100
patience = iters
use_resampling = False
resampling_per_iters = 0
num_max_errors = 1

train_config = {
    "iterations": iters,
    "patience": patience,
    "print_every": 1,
    "num_epochs_adam": num_epochs_adam,
    "num_epochs_lbfgs": num_epochs_lbfgs,
}

# Train baseline
pinn.train(data_config=data_config,
           train_config=train_config,
           use_resampling=use_resampling,
           resampling_per_iters=resampling_per_iters,
           num_max_errors=num_max_errors)